In [ ]:
import os
from dotenv import load_dotenv
import requests
import time

load_dotenv()

API_TOKEN = os.getenv("API_TOKEN")
header = {'Authorization': f'Token {API_TOKEN}'}
signalurl = "https://network.satnogs.org/api/observations/?waterfall_status=1&format=json"
nosignalurl = "https://network.satnogs.org/api/observations/?waterfall_status=0&format=json"
NUM_PAGES = 800

In [ ]:
prepare_directores = True
if prepare_directores:
    os.makedirs("data", exist_ok=True)
    os.makedirs("data/with_waterfall", exist_ok=True)
    os.makedirs("data/without_waterfall", exist_ok=True)

In [ ]:
link = signalurl

def download_waterfalls(url, folder, num_pages):
    link = url
    for i in range(num_pages - 1):
        response = requests.get(link, headers=header)
        if response.status_code == 429:
            print("Rate limit exceeded. Waiting...")
            time.sleep(60)
            response = requests.get(link, headers=header)

        observation_ids = [k['id'] for k in response.json()]
        waterfall_urls = [k['waterfall'] for k in response.json()]
        print(f"Page {i+1}: Downloading {len(waterfall_urls)} waterfalls")
        for obs_id, waterfall_url in zip(observation_ids, waterfall_urls):
            waterfall_data = requests.get(waterfall_url).content
            with open(f"{folder}/obs_{obs_id}.png", "wb") as f:
                f.write(waterfall_data)
        link = response.links.get('next', {}).get('url', None)
        if link is None:
            break

In [ ]:
download_waterfalls(signalurl, "data/with_waterfall", NUM_PAGES)

In [ ]:
download_waterfalls(nosignalurl, "data/without_waterfall", NUM_PAGES)